# Terry Real Corpus Processing

**Purpose**: Process Terry Real's 3 books into ChromaDB collection for RAG-enhanced AI conversations

**Task 2 Requirements**:
- 📚 Extract text from Terry Real PDFs systematically
- 🔪 Implement semantic chunking for relationship concepts
- 🏷️ Preserve metadata (book source, chapter, concept type)
- 🚀 Batch embed all chunks with validated all-MiniLM-L6-v2
- ✅ Validate quality - chunk coherence and embedding coverage

**Technology Stack**: ChromaDB + all-MiniLM-L6-v2 (validated in Task 1)

---

## 📋 Processing Overview

**Source Materials**:
1. `terry-real-how-can-i-get-through-to-you.pdf`
2. `terry-real-new-rules-of-marriage.pdf`
3. `terry-real-us-getting-past-you-and-me.pdf`

**Processing Pipeline**:
1. **Text Extraction** - Extract clean text from PDFs
2. **Content Analysis** - Understand structure and identify chapters
3. **Chunking Strategy** - Semantic chunking for relationship concepts
4. **Metadata Creation** - Preserve book/chapter/concept information
5. **Embedding Generation** - Process with all-MiniLM-L6-v2
6. **Quality Validation** - Test retrieval and coherence
7. **Performance Testing** - Verify query performance for AI conversations

---

## 1. Dependencies & Environment Setup

In [9]:
# Core dependencies
import os
import re
import time
from pathlib import Path

# PDF processing
from pdfminer.high_level import extract_text

# Text processing and chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ChromaDB and embeddings
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

# Data analysis and visualization
import pandas as pd
import numpy as np
from collections import Counter

print("📦 All dependencies imported successfully")
print(f"ChromaDB version: {chromadb.__version__}")

📦 All dependencies imported successfully
ChromaDB version: 1.0.12


In [10]:
# Project configuration
PROJECT_ROOT = Path("../..").resolve()  # From notebooks/ to project root
PDF_DIR = PROJECT_ROOT / "docs" / "Research" / "source-materials" / "pdf books"
CHROMA_DIR = PROJECT_ROOT / "rag_dev" / "chroma_db"
COLLECTION_NAME = "terry_real_corpus"

# Processing parameters (we'll optimize these)
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
EMBEDDING_MODEL = "all-MiniLM-L6-v2"  # Validated in Task 1

print(f"📁 PDF Directory: {PDF_DIR}")
print(f"📁 ChromaDB Directory: {CHROMA_DIR}")
print(f"🗂️ Collection Name: {COLLECTION_NAME}")
print(f"🔧 Chunk Size: {CHUNK_SIZE}, Overlap: {CHUNK_OVERLAP}")
print(f"🤖 Embedding Model: {EMBEDDING_MODEL}")

# Verify PDF files exist
pdf_files = list(PDF_DIR.glob("*.pdf"))
print(f"\n📚 Found {len(pdf_files)} PDF files:")
for pdf in pdf_files:
    print(f"   - {pdf.name}")
    
if len(pdf_files) != 3:
    print("⚠️ Expected 3 Terry Real PDFs, please verify file paths")
else:
    print("✅ All Terry Real PDFs found")

📁 PDF Directory: D:\Github\Relational_Life_Practice\docs\Research\source-materials\pdf books
📁 ChromaDB Directory: D:\Github\Relational_Life_Practice\rag_dev\chroma_db
🗂️ Collection Name: terry_real_corpus
🔧 Chunk Size: 1000, Overlap: 200
🤖 Embedding Model: all-MiniLM-L6-v2

📚 Found 3 PDF files:
   - terry-real-how-can-i-get-through-to-you.pdf
   - terry-real-new-rules-of-marriage.pdf
   - terry-real-us-getting-past-you-and-me.pdf
✅ All Terry Real PDFs found


In [11]:
# Initialize ChromaDB client and embedding model
print("🚀 Initializing ChromaDB and embedding model...")

# Create ChromaDB directory if it doesn't exist
CHROMA_DIR.mkdir(parents=True, exist_ok=True)

# Initialize persistent ChromaDB client
client = chromadb.PersistentClient(path=str(CHROMA_DIR))
print(f"✅ ChromaDB client initialized at {CHROMA_DIR}")

# Initialize embedding model (same as Task 1 validation)
embedder = SentenceTransformer(EMBEDDING_MODEL)
print(f"✅ Embedding model '{EMBEDDING_MODEL}' loaded")
print(f"📐 Embedding dimension: {embedder.get_sentence_embedding_dimension()}")

# Verify this matches our Task 1 validation (should be 384)
expected_dim = 384
actual_dim = embedder.get_sentence_embedding_dimension()
if actual_dim == expected_dim:
    print(f"✅ Embedding dimensions match Task 1 validation: {actual_dim}")
else:
    print(f"⚠️ Dimension mismatch! Expected {expected_dim}, got {actual_dim}")

🚀 Initializing ChromaDB and embedding model...
✅ ChromaDB client initialized at D:\Github\Relational_Life_Practice\rag_dev\chroma_db
✅ Embedding model 'all-MiniLM-L6-v2' loaded
📐 Embedding dimension: 384
✅ Embedding dimensions match Task 1 validation: 384


In [13]:
# Clean up any existing collection (for fresh processing)
print(f"🧹 Preparing clean environment for {COLLECTION_NAME}...")

try:
    existing_collection = client.get_collection(COLLECTION_NAME)
    client.delete_collection(COLLECTION_NAME)
    print(f"🗑️ Deleted existing collection '{COLLECTION_NAME}'")
except Exception as e:
    print(f"ℹ️ No existing collection to delete: {e}")

# Create fresh collection
collection = client.create_collection(
    name=COLLECTION_NAME,
    metadata={"description": "Terry Real's Relational Life Therapy corpus for AI conversations"}
)
print(f"✅ Fresh collection '{COLLECTION_NAME}' created")
print(f"📊 Collection count: {collection.count()} documents")

print("\n" + "="*60)
print("🎉 ENVIRONMENT SETUP COMPLETE")
print("✅ Dependencies loaded")
print("✅ Paths configured and verified")
print("✅ ChromaDB client initialized")
print("✅ Embedding model ready (384 dimensions)")
print("✅ Fresh collection created")
print("🚀 Ready for PDF text extraction")
print("="*60)

🧹 Preparing clean environment for terry_real_corpus...
🗑️ Deleted existing collection 'terry_real_corpus'
✅ Fresh collection 'terry_real_corpus' created
📊 Collection count: 0 documents

🎉 ENVIRONMENT SETUP COMPLETE
✅ Dependencies loaded
✅ Paths configured and verified
✅ ChromaDB client initialized
✅ Embedding model ready (384 dimensions)
✅ Fresh collection created
🚀 Ready for PDF text extraction


## 2. PDF Text Extraction & Content Analysis

**Objective**: Extract and analyze text from Terry Real PDFs to understand structure and optimize chunking strategy

**Steps**:
1. Test text extraction from one book
2. Analyze content structure and chapter organization  
3. Identify patterns for semantic chunking
4. Validate text quality and readability

### Code Cell 1: Test Single PDF Extraction

In [14]:
# Test extraction from one Terry Real book first
print("🔍 Testing PDF text extraction...")

# Select first PDF for testing
test_pdf = pdf_files[0]
print(f"📖 Testing with: {test_pdf.name}")

# Extract text from PDF
start_time = time.time()
raw_text = extract_text(str(test_pdf))
extraction_time = time.time() - start_time

print(f"⏱️ Extraction time: {extraction_time:.2f} seconds")
print(f"📊 Total characters: {len(raw_text):,}")
print(f"📊 Total lines: {len(raw_text.splitlines()):,}")

# Show first 1000 characters to understand structure
print("\n" + "="*60)
print("📋 FIRST 1000 CHARACTERS:")
print("="*60)
print(raw_text[:1000])
print("="*60)

🔍 Testing PDF text extraction...
📖 Testing with: terry-real-how-can-i-get-through-to-you.pdf
⏱️ Extraction time: 17.15 seconds
📊 Total characters: 579,103
📊 Total lines: 12,212

📋 FIRST 1000 CHARACTERS:
How Can I Get Through to You?: Closing the
Intimacy Gap Between Men and Women

Terrence Real

2003

1

How Can I Get Through to You?

Reconnecting Men and Womeng

Terrence Real

SCRIBNER
New York London Toronto Sydney Singapore

SCRIBNER
1230 Avenue of the Americas
New York, NY 10020
www.SimonandSchuster.com

2

Copyright © 2002 by Terrence Real

All rights reserved, including the right of reproduction in whole or in part in
any form.

SCRIBNER and design are trademarks of Macmillan Library Reference USA,
Inc., used under license by Simon & Schuster, the publisher of this work.

For information about special discounts for bulk purchases, please contact Simon
& Schuster Special Sales: 1-800-465-6798 or business@simonandschuster.com

DESIGNED BY ERICH HOBBING

Text set in Janson

Manufa

### Code Cell 2: Content Structure Analysis

In [ ]:
# Analyze text structure to understand Terry Real's organization
print("🔍 Analyzing content structure...")

lines = raw_text.splitlines()
non_empty_lines = [line.strip() for line in lines if line.strip()]

print(f"📊 Non-empty lines: {len(non_empty_lines):,}")

# Look for chapter patterns
chapter_patterns = [
    r"^Chapter\s+\d+",
    r"^CHAPTER\s+\d+", 
    r"^\d+\.\s+",
    r"^Part\s+\w+",
    r"^PART\s+\w+"
]

potential_chapters = []
for i, line in enumerate(non_empty_lines[:200]):  # Check first 200 lines
    for pattern in chapter_patterns:
        if re.match(pattern, line, re.IGNORECASE):
            potential_chapters.append((i, line))

print(f"\n📚 Potential chapter markers found: {len(potential_chapters)}")
for i, chapter in enumerate(potential_chapters[:10]):  # Show first 10
    line_num, text = chapter
    print(f"   {i+1}. Line {line_num}: {text[:80]}...")

# Look for other structural elements
print(f"\n🔍 Analyzing structural patterns...")

# Count lines with different characteristics
short_lines = [line for line in non_empty_lines if len(line) < 50]
long_lines = [line for line in non_empty_lines if len(line) > 200]
all_caps_lines = [line for line in non_empty_lines if line.isupper() and len(line) > 5]

print(f"📏 Short lines (<50 chars): {len(short_lines):,}")
print(f"📏 Long lines (>200 chars): {len(long_lines):,}")
print(f"📢 All caps lines: {len(all_caps_lines):,}")

# Show some examples
if all_caps_lines:
    print(f"\n📢 Sample all-caps lines (potential headings):")
    for line in all_caps_lines[:5]:
        print(f"   {line}")

🔍 Analyzing content structure...
📊 Non-empty lines: 9,025

📚 Potential chapter markers found: 17
   1. Line 70: 1. Love on the Ropes : Men and Women in Crisis...
   2. Line 71: 2. Echo Speaks: Empowering the Woman...
   3. Line 72: 3. Bringing Men in from the Cold...
   4. Line 73: 4. Psychological Patriarchy: The Dance of Contempt...
   5. Line 74: 5. The Third Ring: A Conspiracy of Silence...
   6. Line 75: 6. The Unspeakable Pain of Collusion...
   7. Line 76: 7. Narcissus Resigns: An Unconventional Therapy...
   8. Line 77: 8. Small Murders : How We Lose Passion...
   9. Line 78: 9. A New Model of Love...
   10. Line 79: 10. Recovering Real Passion...

🔍 Analyzing structural patterns...
📏 Short lines (<50 chars): 2,271
📏 Long lines (>200 chars): 1
📢 All caps lines: 60

📢 Sample all-caps lines (potential headings):
   SCRIBNER
   SCRIBNER
   DESIGNED BY ERICH HOBBING
   ISBN-10: 0-684-86877-6
   ISBN-13: 978-0-684-86877-6


### Code Cell 3: Content Quality Assessment

In [ ]:
# Assess text quality and identify any extraction issues
print("🔍 Assessing text extraction quality...")

# Sample paragraphs for readability check
sample_paragraphs = []
current_paragraph = []

for line in non_empty_lines[:500]:  # Check first 500 lines
    if len(line) > 100:  # Likely paragraph content
        current_paragraph.append(line)
    elif current_paragraph:
        if len(current_paragraph) >= 2:  # Multi-line paragraph
            sample_paragraphs.append(" ".join(current_paragraph))
        current_paragraph = []
    
    if len(sample_paragraphs) >= 3:  # Get 3 sample paragraphs
        break

print(f"📖 Sample readable paragraphs found: {len(sample_paragraphs)}")

for i, paragraph in enumerate(sample_paragraphs):
    print(f"\n📖 Sample Paragraph {i+1} ({len(paragraph)} chars):")
    print("-" * 50)
    print(paragraph[:300] + ("..." if len(paragraph) > 300 else ""))
    print("-" * 50)

# Check for common extraction issues
issues = []
if raw_text.count("�") > 0:
    issues.append(f"Encoding issues: {raw_text.count('�')} replacement characters")

if len([line for line in lines if len(line) == 1]) > 100:
    issues.append("Many single-character lines (possible formatting issues)")

if len(re.findall(r'\w+\w+\w+', raw_text)) < len(raw_text.split()) * 0.8:
    issues.append("Possible word separation issues")

if issues:
    print(f"\n⚠️ Potential extraction issues:")
    for issue in issues:
        print(f"   - {issue}")
else:
    print(f"\n✅ Text extraction quality looks good!")

### Code Cell 4: Chunking Strategy Analysis

In [ ]:
# Analyze optimal chunking strategy for Terry Real content
print("🔪 Analyzing optimal chunking strategy...")

# Test current chunking parameters
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Create initial chunks for analysis
test_chunks = splitter.split_text(raw_text[:50000])  # Test with first 50k characters

print(f"📊 Test chunking results:")
print(f"   Source text: {50000:,} characters")
print(f"   Generated chunks: {len(test_chunks):,}")
print(f"   Average chunk size: {np.mean([len(chunk) for chunk in test_chunks]):.0f} characters")
print(f"   Min chunk size: {min(len(chunk) for chunk in test_chunks)}")
print(f"   Max chunk size: {max(len(chunk) for chunk in test_chunks)}")

# Show sample chunks
print(f"\n📋 Sample chunks for quality assessment:")
for i, chunk in enumerate(test_chunks[:3]):
    print(f"\n--- Chunk {i+1} ({len(chunk)} chars) ---")
    print(chunk[:200] + ("..." if len(chunk) > 200 else ""))
    print("--- End Chunk ---")

# Analyze chunk coherence
relationship_terms = [
    "relationship", "marriage", "partner", "couple", "intimacy", 
    "communication", "conflict", "emotion", "boundary", "repair",
    "empathy", "connection", "trust", "vulnerability", "healing"
]

chunks_with_terms = []
for chunk in test_chunks[:20]:  # Analyze first 20 chunks
    term_count = sum(1 for term in relationship_terms if term.lower() in chunk.lower())
    chunks_with_terms.append(term_count)

avg_terms_per_chunk = np.mean(chunks_with_terms)
print(f"\n🔍 Relationship content analysis:")
print(f"   Average relationship terms per chunk: {avg_terms_per_chunk:.1f}")
print(f"   Chunks with 3+ relationship terms: {sum(1 for count in chunks_with_terms if count >= 3)}/{len(chunks_with_terms)}")

if avg_terms_per_chunk >= 2:
    print("✅ Good relationship content density in chunks")
else:
    print("⚠️ Consider adjusting chunk size for better content coherence")

### Code Cell 5: Processing Strategy Summary

In [ ]:
# Summarize findings and prepare processing strategy
print("📋 PROCESSING STRATEGY SUMMARY")
print("=" * 60)

print(f"📖 Source Material Analysis:")
print(f"   Book: {test_pdf.name}")
print(f"   Total characters: {len(raw_text):,}")
print(f"   Total lines: {len(raw_text.splitlines()):,}")
print(f"   Extraction time: {extraction_time:.2f} seconds")

print(f"\n🏗️ Content Structure:")
print(f"   Potential chapters found: {len(potential_chapters)}")
print(f"   All-caps headings: {len(all_caps_lines)}")
print(f"   Text quality: {'Good' if not issues else 'Issues detected'}")

print(f"\n🔪 Chunking Strategy:")
print(f"   Current chunk size: {CHUNK_SIZE}")
print(f"   Current overlap: {CHUNK_OVERLAP}")
print(f"   Average chunk size: {np.mean([len(chunk) for chunk in test_chunks]):.0f} chars")
print(f"   Relationship content density: {avg_terms_per_chunk:.1f} terms/chunk")

# Recommendations
print(f"\n💡 Recommendations:")
if len(potential_chapters) > 0:
    print("   ✅ Chapter structure detected - can preserve in metadata")
else:
    print("   ⚠️ No clear chapter structure - use semantic chunking only")

if avg_terms_per_chunk >= 2:
    print("   ✅ Current chunk size preserves relationship content well")
else:
    print("   📝 Consider increasing chunk size for better content coherence")

print(f"\n🚀 Ready to process all {len(pdf_files)} Terry Real books!")
print("=" * 60)